In [17]:
import nbimporter

from geometry_2d import process_images
from add_camera_position import get_df_with_camera_position

import numpy as np
import cv2
from pyproj import Transformer
from scipy.spatial.transform import Rotation as R

def get_3d(data):
    """
    data: dict mapping image_filename -> {
      'leftmost point': (x_px, y_px),
      'lat': float, 'lon': float, 'alt': float
    }
    """

    # 1) Camera intrinsics (DJI M4TD wide) – unchanged
    sensor_width_mm  = 7.6
    sensor_height_mm = 5.7
    focal_length_mm  = 7.0
    image_width_px   = 4032
    image_height_px  = 3024

    fx = (focal_length_mm / sensor_width_mm)  * image_width_px
    fy = (focal_length_mm / sensor_height_mm) * image_height_px
    cx, cy = image_width_px/2, image_height_px/2

    K = np.array([[fx,  0, cx],
                  [ 0, fy, cy],
                  [ 0,  0,  1]])

    # 2) CRS transformers
    to_ecef   = Transformer.from_crs("epsg:4326","epsg:4978",always_xy=True)
    from_ecef = Transformer.from_crs("epsg:4978","epsg:4326",always_xy=True)

    # 3) Nadir rotation
    rot_nadir = R.from_euler('xyz', [0, -90, 0], degrees=True).as_matrix()

    # 4) Build projection matrices
    P_list = []
    for img_name in data:
        values= data[img_name]
        Cx, Cy, Cz = to_ecef.transform(values["lat"], values["lon"], values["alt"])
        C = np.array([[Cx],[Cy],[Cz]])
        Rt = np.hstack((rot_nadir, -rot_nadir @ C))
        P_list.append(K @ Rt)

    # 5) Collect 2D points
    pts = np.array([data[img]["point"] for img in data], dtype=float)
    if pts.shape[0] < 2:
        raise ValueError("Need at least two cameras/points to triangulate")

    p1 = pts[0].reshape(2,1)
    p2 = pts[1].reshape(2,1)

    # 6) Triangulate
    Xh = cv2.triangulatePoints(P_list[0], P_list[1], p1, p2)

    # 7) From homogeneous to ECEF
    X_ecef = (Xh[:3] / Xh[3]).flatten()

    # 8) Back to GPS
    lat,lon,  alt = from_ecef.transform(*X_ecef)

    print(f"Triangulated GPS coordinates: lat={lat:.6f}, lon={lon:.6f}, alt≈{alt:.2f} m")
    return lon, lat, alt

def get_line():
    df = get_df_with_camera_position()
    
    points_to_triangulate = process_images(df,image_width_px=4032, image_height_px=3024)
    
    
    points_on_line = []
    for coord in points_to_triangulate: 
        points_on_line.append(get_3d(coord))
    return points_on_line


get_line()

Triangulated GPS coordinates: lat=49.099393, lon=12.180903, alt≈487.54 m
Triangulated GPS coordinates: lat=49.099392, lon=12.180903, alt≈487.69 m
Triangulated GPS coordinates: lat=49.099391, lon=12.180907, alt≈487.49 m
Triangulated GPS coordinates: lat=49.099390, lon=12.180912, alt≈487.29 m


[(12.18090253874485, 49.099392570576676, 487.5393529124558),
 (12.18090316789037, 49.099392459162466, 487.68679808452725),
 (12.180907204492144, 49.099391322882276, 487.4937693476677),
 (12.180911566568401, 49.0993901935563, 487.2905433224514)]